In [ ]:
import sys
%matplotlib inline
#from psychopy import gui
from matplotlib.mlab import csv2rec
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import leastsq
from pypsignifit import (BootstrapInference,GoodnessOfFit,ParameterPlot,
                         ThresholdPlot)

def weibull(x,threshx,slope,guess,flake,threshy=None):

    if threshy is None:
        threshy = 1-(1-guess)*np.exp(-1)
        
    k = (-np.log( (1-threshy)/(1-guess) ))**(1/slope)
    weib = flake - (flake-guess)*np.exp(-(k*x/threshx)**slope)
    return weib 
 
def weib_fit(pars):
    thresh,slope = pars
    
    return weibull(x,thresh,slope,guess,flake)
    
def err_func(pars):
    return y-weib_fit(pars)

if __name__=="__main__":

    bootstrap_n = 1000

    #Weibull params:
    guess = 0.5 #The guessing rate is 0.5
    flake = 0.99
    slope = 3.5
    #file_name = str(gui.fileOpenDlg(tryFilePath='./data')[0])
    file_name='./data/SS_kelly_09082015_6.psydat'
    file_read = file(file_name,'r')
    p = {} #This will hold the params
    l = file_read.readline()

    while l[0]=='#':
        try:
            p[l[1:l.find(':')-1]]=float(l[l.find(':')+1:l.find('\n')]) 

        #Not all the parameters can be cast as float (the task and the
        #subject): 
        except:
            p[l[2:l.find(':')-1]]=l[l.find(':')+1:l.find('\n')]

        l = file_read.readline()
    data_rec = csv2rec(file_name)
    annulus_target_contrast = data_rec['annulus_target_contrast']-p[' annulus_contrast']
    contrast = data_rec['annulus_target_contrast']-p[' annulus_contrast']
    correct = data_rec['correct']   

    #Which staircase to analyze:
    if p['task'] == ' Fixation ':
        contrast_all = data_rec['fixation_target_contrast']
    elif p['task']== ' Annulus ':
        contrast_all = data_rec['annulus_target_contrast']-p[' annulus_contrast']

    labelit = ['annulus_off','annulus_on']
    if p[' surround_contrast']%180 == 0 :
        print 'Surround: vertical'
    else:
        print 'Surround: horizontal'
       
    if p[' annulus_contrast']%180 == 0 :
        print 'Annulus: vertical'
    else:
        print 'Annulus: horizontal'
       

    if p['SurroundType'] is 'yes':
        print 'Surround contrast: %1.1f' % (p[' surround_contrast'])
    else:
        print 'Surround contrast: 0'

 
    #Switch on the two annulus tasks (annulus on vs. annulus off):
#    for idx_annulus,operator in enumerate(['<','>=']):
    hit_amps = contrast[correct==1]
    miss_amps = contrast[correct==0]
    all_amps = np.hstack([hit_amps,miss_amps])
        #Get the data into this form:
        #(stimulus_intensities,n_correct,n_trials)
    stim_intensities = np.unique(all_amps)
    n_correct = [len(np.where(hit_amps==i)[0]) for i in stim_intensities]
    n_trials = [len(np.where(all_amps==i)[0]) for i in stim_intensities]

    Data = zip(stim_intensities,n_correct,n_trials)

    x = []
    y = []

#        for idx,this in enumerate(Data):
            #Take only cases where there were at least 3 observations:
#    labelit = p['annulus_ori'] 
    for idx,this in enumerate(Data):
        if n_trials>=3:
    
        #Contrast values: 
            x = np.hstack([x,this[2] * [this[0]]])
                #% correct:
            y = np.hstack([y,this[2] * [this[1]/float(this[2])]])
    
    initial = np.mean(x),slope
    this_fit , msg = leastsq(err_func,initial)
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(x,y,'o')
    x_for_plot = np.linspace(np.min(x),np.max(x),100)
    ax.plot(x_for_plot,weibull(x_for_plot,this_fit[0],
                                   this_fit[1],
                                   guess,
                                   flake))
    ax.set_title('%s:thresh=%1.2f::slope=%1.2f'
                     %(p['task'],this_fit[0],this_fit[1]))
    ax.set_xlabel('Percentage correct')
    ax.set_ylabel('Target contrast - annulus contrast')
    file_stem = file_name.split('/')[-1].split('.')[0]
    fig.savefig('%s.png'%(file_stem))
            
 

       
    bootstrap_th = []
    bootstrap_slope = []
    keep_x = x
    keep_y = y
    keep_th = this_fit[0]
    for b in xrange(bootstrap_n):
            b_idx = np.random.randint(0,x.shape[0],x.shape[0])
            x = keep_x[b_idx]
            y = keep_y[b_idx]
            initial = np.mean(x),slope
            this_fit , msg = leastsq(err_func,initial)
            bootstrap_th.append(this_fit[0])
            bootstrap_slope.append(this_fit[0])
    upper = np.sort(bootstrap_th)[bootstrap_n*0.975]
    lower = np.sort(bootstrap_th)[bootstrap_n*0.025]
    print "Threshold estimate: %s, CI: [%s,%s]"%(keep_th,
                                                                    lower,
                                                                    upper)
            

In [14]:
    #Both tasks are honest-to-god 2AFC:
    constraints = ( 'unconstrained', 'unconstrained', 'unconstrained') 

    B = BootstrapInference ( Data, priors=constraints, nafc=2 )
    B.sample()
    print 'Threshold: %s'%(B.getThres())
    print 'Slope: %s' % (B.getSlope())
    print 'CI: [%s,  %s]'%(B.getCI(1)[0],
                           B.getCI(1)[1])
    print 'Last value in the staircase: %s'%contrast[-1]

Threshold: 0.0753935860058
Slope: 247.355769414
CI: [0.0664834849713,  0.0947029699904]
Last value in the staircase: 0.066666666667


In [16]:
bootstrap_th


[0.10310490451016457,
 0.104505354915419,
 0.10326898287942746,
 0.10707133535281779,
 0.099577227879016766,
 0.10568280647201601,
 0.10552995869860106,
 0.10300048732479627,
 0.1062885340755744,
 0.1022006077389648,
 0.1028497834165535,
 0.10539844068247224,
 0.10335726930072808,
 0.10585050354776655,
 0.10910066706071934,
 0.10516527537601442,
 0.10715358939729733,
 0.10487988927386784,
 0.10511530494159564,
 0.10816188681053429,
 0.10996308343223832,
 0.1061371515097424,
 0.10335733511627897,
 0.10873684500970233,
 0.10093073261536371,
 0.10677653620305595,
 0.10273799747247889,
 0.10617005339336401,
 0.110871171574008,
 0.10164899015769445,
 0.10659592296300428,
 0.10541032222058636,
 0.10436342563183733,
 0.10301137521295683,
 0.10533320296055514,
 0.10365021125130208,
 0.10381842981895634,
 0.103093063501979,
 0.10658260272087798,
 0.10566307684020101,
 0.10655528931419322,
 0.106633560641376,
 0.10376270624124993,
 0.10806414133154445,
 0.10602056762983032,
 0.10707833318369286,